In [35]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

oai_client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

df=pd.read_pickle('cleaned_quests_with_glossary.pkl')
df.head()

,id,expansion,title_en,objective_en,description_en,progress_en,completion_en,title_uk,objective_uk,description_uk,progress_uk,completion_uk,cat,glossary
0,2,classic,Sharptalon's Claw,Bring Sharptalon's Claw to Senani Thunderheart...,The mighty hippogryph Sharptalon has been slai...,"Yes, mighty <class>, I sensed your arrival. I ...","Most impressive, <name>... the claw of Sharpta...",Кіготь Гостропазура,Принесіть кіготь Гостропазура до Сенані Грозов...,"Могутнього Гостропазура вбито, а кіготь гіпогр...","Так, {стать:могутній:могутня} {клас:к}, я відч...","Я вражена, {ім'я:к}... кіготь Гостропазура — н...",Kalimdor/Ashenvale,"[{'en': 'Ashenvale', 'uk': 'Ясенеділ'}, {'en':..."
1,5,classic,Jitters' Growling Gut,Speak with Chef Grual.,I've been stuck hiding in this ghost town for ...,None,"You need some Crab Cakes, do you? Well I might...",Бурчання Тремтінієвого живота,Поговоріть з шеф-кухарем Ґруалем.,Я вже декілька тижнів ховаюсь в цьому містечку...,None,"Хочеш крабових пиріжків? Ну, я можу приготуват...",Eastern Kingdoms/Duskwood,"[{'en': 'cook', 'uk': 'кухар/кухарка'}, {'en':..."
2,6,classic,Bounty on Garrick Padfoot,Kill Garrick Padfoot and bring his head to Dep...,Garrick Padfoot - a cutthroat who's plagued ou...,Did you find Garrick's shack? Are we finally f...,Hah - you caught him! You've done Elwynn a gre...,Винагорода за голову Ґарріка М'якоступа,Убийте Ґарріка М'якоступа та принесіть його го...,"Ґаррік М'якоступ — головоріз, який мучить наши...",Ти {стать:знайшов:знайшла} халупу Ґарріка? Нев...,Ха-а — ти {стать:спіймав:спіймала} його! Ти {с...,Eastern Kingdoms/Elwynn Forest,"[{'en': 'deputy', 'uk': 'заступник'}, {'en': '..."
3,7,classic,Kobold Camp Cleanup,"Kill 10 Kobold Vermin, then return to Marshal ...","Your first task is one of cleansing, <name>. A...","How goes the hunting, <name>? Have you found a...","Well done, citizen. Those kobolds are thieves ...",Зачистка табору кобольдів,"Убийте 10 кобольдів-шкідників, а потім поверні...",Твоє перше завдання буде пов'язане із зачищенн...,"Як там твоє полювання, {ім'я:к}? Ти {стать:зна...","Молодець, {стать:громадянине:громадянко}. Ці к...",Eastern Kingdoms/Elwynn Forest,"[{'en': 'kobold', 'uk': 'кобольд'}, {'en': 'St..."
4,8,classic,A Rogue's Deal,Deliver the Nondescript Letter to Innkeeper Re...,"Hey, mate! Do a favor for a young man who's be...",Yes? Yes? What is it?\n\nThe forsaken; they ce...,Oh? A letter? Wonderful news! I've been waitin...,Угода пройдисвіта,Доставте непідписаний лист корчмарці Рені в Ті...,"Гей, {стать:приятелю:приятелько}! Зроби-но пос...","Так? Так? Що це?\n\nВідречені. У них, безумовн...",Га? Лист? Гарні новини! Я так чекала на звістк...,Eastern Kingdoms/Tirisfal Glades,"[{'en': 'innkeeper', 'uk': 'корчмар/корчмарка'..."


In [36]:
translated = df[df['title_uk'].notnull()]
untranslated = df[df['title_uk'].isnull()]

In [ ]:
TRANSLATION_SYSTEM_PROMPT = """You are a professional translator specializing in MMORPG game localization, responsible for translating World of Warcraft content from English to Ukrainian. 
You will receive a JSON file containing an "original" section with the object to translate. 
Your task is to respond with a JSON containing single section "translation" that replicates the structure of the "original" section, replacing source text with translated values while maintaining all keys.
Use your knowledge of the Warcraft universe to ensure accuracy and immersion.

### **Guidelines**:

1. **Preserve Structure**:
   - The output JSON must retain the exact structure of the "original" section.
   - Ensure all keys remain the same while replacing values with their Ukrainian translations.

2. **Maintain Placeholders**:
   - Always replace name/race/class placeholders in the original text with curly-bracedanslated placeholders with shortened Ukrainian grammatical case after a semicolon. For `<name>`, `<race>`, `<class>` use `{ім'я:н}`, `{раса:д}`, `{клас:к}` with correct grammatical case afet semicolon.

3. **Adapt for Gender**:
   - Translate text to be suitable for both genders. For placeholders with gender-specific variations (e.g., `<his/her>`, `<priest/priestess>`) and translated words that have different gender forms use the format `{стать:male:female}` (e.g., `{стать:його:її}`, `{стать:жрець:жриця}`).
   - Gender placeholders should contain full words, so don't break words with placeholder: for example, instead of кмітлив{стать:ий:а} use {стать:кмітливий:кмітлива}
   - Gender placeholders can contain more that one word. If there's several consequent gender-specific placeholders or they are divided with small word - put tham in single placeholder. Fo example: instead of "дуже {стать:достойний:достойна} та {стать:спроможний:спроможна}" use "дуже {стать:достойний та спроможний:достойна та спроможна}"

4. **Formal and Informal Tone**:
   - **Quest `objective` field**: Translate using a formal tone, where "you" becomes "ви" and "your" becomes "ваш."
   - **Quest `description`, `progress`, and `completion` fields**: By default, translate these fields using an informal tone, where "you" becomes "ти" and "your" becomes "твій.". If text describes environment/interaction or NPC speaks to player politely using "sir", "lady" etc - use formal tone.

5. **Preserve Descriptive Formatting**:
   - Retain all text enclosed in angle brackets (`<>`) that describes the environment, character reactions, or similar narrative elements. Translate the content within the brackets but preserve their formatting.

6. **Maintain Lore and Style**:
   - Ensure the translation captures the epic, immersive tone of World of Warcraft.
   - Use established Warcraft terminology and lore-consistent language in Ukrainian.

7. **Glossaries and terminology**:
   - Always strictly refer to the translation glossary. Always use terms' translations from glossary, even if other more commonly used or recognized translations are available.
   - Use consistent terminology throughout all translations. Adhere strictly to any provided glossaries, ensuring uniformity in translating terms such as character names, locations, and game-specific items.

8. **Readability**:
   - Ensure translations are natural, engaging, and free from awkward phrasing.
   - Text should feel native and immersive for Ukrainian-speaking players.

---

### **Input Example**:

```json
{
  "original": {
    "quest": {
      "title": "Get rid of these boars",
      "objective": "Kill 12 boars and return to John at Scarlet Monastery in Silverpine Forest.",
      "description": "Reports have come in that wild boars trample our crops. We thought that other adventurer had dealt with this, <class>!\n\nI suggest that you to teach them a lesson as proof of your devotion.",
      "progress": "Have you clear our fields of wild boars?",
      "completion": "<John nods to you.>\n\nWe are pleased that you have prooved your devition, <name>. Surely now that you have made our point clear multiple times, their senseless attacks upon our crops will cease?"
    }
  }
}
```

---

### **Output Example**:

```json
{
  "translation": {
    "quest": {
      "title": "Позбудься цих вепрів",
      "objective": "Вбийте 12 вепрів і поверніться до Джона в Багряний Монастир, що в Срібнохвойному пралісі.",
      "description": "Надходять повідомлення про диких вепрів, що топчуть наші посіви. Ми думали, що інші шукачі пригод вже розібралися з цим, {клас:к}!\n\nРаджу тобі провчити їх як доказ твоєї відданості.",
      "progress": "Ти вже {стать:очистив:очистила} наші поля від диких вепрів?",
      "completion": "<Джон киває вам.>\n\nМи задоволені, що ти {стать:довів:довела} свою відданість, {ім'я:к}. Тепер, коли ми вже неодноразово довели свою точку зору, може їхні безглузді напади на наші посіви припиняться?"
    }
  }
}
```

---
"""

In [36]:
def generate_request_json(row):
    return {
        "original": {
            "quest": {
                "title": row["title_en"],
                "objective": row["objective_en"],
                "description": row["description_en"],
                "progress": row["progress_en"],
                "completion": row["completion_en"]
            }
        },
        "glossary": {
            "glossary": row["glossary"]
        }
    }

def translate_row(row):
    import json
    messages = [{'role': 'system', 'content': TRANSLATION_SYSTEM_PROMPT}, {'role': 'user', 'content': str(generate_request_json(row))}]
    response = oai_client.chat.completions.create(
    model="ft:gpt-4o-2024-08-06:personal:classicua-v5:AXuONZ0V",
    messages=messages,
    response_format={"type":"json_object"},
    temperature=0.7,
    top_p=1
    )
    json_response = response.choices[0].message.content
    translation = json.loads(json_response)['translation']['quest']
    print(translation)
    return {
            "title_uk": translation.get("title", None),
            "objective_uk": translation.get("objective", None),
            "description_uk": translation.get("description", None),
            "progress_uk": translation.get("progress", None),
            "completion_uk": translation.get("completion", None),
    }


In [38]:
untranslated[untranslated['expansion']=='sod'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 115 entries, 4729 to 4944
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              115 non-null    int64 
 1   expansion       115 non-null    object
 2   title_en        115 non-null    object
 3   objective_en    112 non-null    object
 4   description_en  113 non-null    object
 5   progress_en     58 non-null     object
 6   completion_en   108 non-null    object
 7   title_uk        0 non-null      object
 8   objective_uk    0 non-null      object
 9   description_uk  0 non-null      object
 10  progress_uk     0 non-null      object
 11  completion_uk   0 non-null      object
 12  cat             115 non-null    object
 13  glossary        115 non-null    object
dtypes: int64(1), object(13)
memory usage: 13.5+ KB


In [28]:
filtered_df = untranslated[untranslated['expansion']=='sod']
translation = filtered_df.apply(translate_row, axis=1)
translation_df = pd.DataFrame(translation.tolist(), index=filtered_df.index)

untranslated.update(translation_df)

{'title': 'У скруті', 'objective': 'Заберіть меч зброєносця Катберта на полі на півдні та поверніть його йому.', 'description': 'Ти мене мало до смерті не налякав! Я почув, як хтось підходить, і був переконаний, що мертві нарешті мене вистежили! І все одно, я радий тебе бачити. Будь ласка, мені потрібно втекти звідси та знайти свого {стать:майстра:майстриню}, {стать:пана:пані} Дальтона. Чи можеш ти принести мою зброю? Її вибили з моїх рук на полі, і мені довелося тікати в цей сарай.', 'progress': 'Ти {стать:знайшов:знайшла} меч?', 'completion': 'Дякую, {клас:к}. Я не можу висловити всю свою вдячність за твою допомогу.\n\nМені потрібно лише трохи часу, щоб зібратися з думками. Я отримав жахливий удар по голові та поки не впевнений, що можу нормально битися. О, світло, допоможи мені, я сподіваюся, що {стать:пан:пані} Дальтон {стать:в безпеці:в безпеці}.'}
{'title': 'Полеглий лицар', 'objective': 'Ведіть зброєносця Катберта до тіла полеглого лицаря, яке ви раніше знайшли, та допоможіть йо

In [41]:
untranslated[untranslated['title_uk'].notnull()]

,id,expansion,title_en,objective_en,description_en,progress_en,completion_en,title_uk,objective_uk,description_uk,progress_uk,completion_uk,cat,glossary
4729,83808,sod,In a Bind,Retrieve Squire Cuthbert's sword in the field ...,You nearly scared me to death! I heard you com...,Have you retrieved the blade?,"Thank you, <class>. I can't express my gratitu...",У скруті,Заберіть меч зброєносця Катберта на полі на пі...,"Ти мене мало до смерті не налякав! Я почув, як...",Ти {стать:знайшов:знайшла} меч?,"Дякую, {клас:к}. Я не можу висловити всю свою ...",Classes/Paladin,"[{'en': 'master', 'uk': 'майстер'}, {'en': 'To..."
4730,83822,sod,The Fallen Knight,Lead Squire Cuthbert to the body of the Fallen...,You say you found his... body? No... my lord c...,None,"It seems I am again in your debt, <class>. Eve...",Полеглий лицар,Ведіть зброєносця Катберта до тіла полеглого л...,"Ти {стать:сказав:сказала}, що знайш{стать:ов:л...",None,"Здається, я знову в боргу перед тобою, {клас:к...",Classes/Paladin,"[{'en': 'lord', 'uk': 'лорд'}, {'en': 'Dalson'..."
4731,83823,sod,A Lesson in Violence,Your squire has grown in skill adventuring alo...,"Your technique is very interesting,<name>. A b...",Your technique is most impressive! A bit rough...,Your squire has grown in skill adventuring alo...,Урок насильства,Ваш {стать:зброєносець:зброєносиця} {стать:ста...,"Твоя техніка дуже цікава, {ім'я:к}. Трохи груб...","Твоя техніка дуже вражаюча! Трохи грубуватіша,...",Ваш {стать:зброєносець:зброєносиця} {стать:ста...,Classes/Paladin,[]
4732,83934,sod,Show Me The Money!,Bring a Tarnished Undermine Real to Pix Xizzix...,"Heya! You look like the ""resourceful adventure...","Whatcha got for me, <race>?",Hey! That's great! \n\nIf you find anymore of ...,Покажи мені гроші!,Принесіть занедбаний піастр Підштольні Піксові...,"Привіт! Ти виглядаєш як ""кмітлив{стать:ий:а} ш...","Що ти {стать:маєш:маєш} для мене, {раса:к}?","О, круто!\n\nЯкщо знайдеш більше наших монет, ...",Kalimdor/Desolace,"[{'en': 'Booty Bay', 'uk': 'Грабіжна Бухта'}, ..."
4733,83935,sod,Clearing the Path,"Kill 5 Blighted Zombies, 10 Skeletal Terrors, ...","The pounding in my head has subsided somewhat,...",The coast is clear?,"Thank you again, <class>. If we had someone li...",Розчищення шляху,"Убийте 5 гнилих зомбі, 10 скелетних жахів і 10...","Біль в голові вщухла, але я не думаю, що зможу...",Усе чисто?,"Ще раз дякую, {клас:к}. Якби з нами був хтось ...",Classes/Paladin,"[{'en': 'Hated', 'uk': 'ненависна'}, {'en': 'S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4940,85555,sod,Heavy Metal,Find someone who can work with exotic metals a...,The large flat disc of elementium you hold bef...,What's that you've got there?,Wow! That's quite the hunk of metal you got th...,Важкий метал,"Знайдіть того, хто вміє працювати з екзотичним...","Великий плоский диск елементію в твоїх руках, ...",Що це в тебе?,"Вау! Оце так шмат металу, {клас:к}.\n\nЕлемент...",Classes/Shaman,"[{'en': 'Blackrock Mountain', 'uk': 'Чорноскел..."
4941,85556,sod,A Particular Set of Skills,Find a way to get into Blackwing Lair and spea...,"Krixix is a genius for sure, but he may not be...","Shouldn't you be working, bub?","Whaddya want, can't you see I'm busy here? Wai...",Певний набір навичок,Знайдіть спосіб потрапити в лігво Чорного Крил...,"Кріксікс безперечно геній, але він може бути н...",Ти ж не байдикувати сюди {стать:прийшов:прийшл...,"Чого тобі треба? Не бачиш, що я зайнятий? Хоча...",Classes/Shaman,"[{'en': 'master', 'uk': 'майстер'}, {'en': 'Bl..."
4942,85557,sod,Efficiency Is Priority One,Find a suitable power source to keep Master El...,"Actually, I just gave myself an idea. You see,...",What are you talkin' to me for? Scram.,Wow! This is perfect!\n\nYou'd think that Vick...,Ефективність — понад усе,"Знайдіть відповідне джерело енергії, щоб кузня...","Знаєте, в мене щойно з'явилася ідея. Бачите, н...",Чого ти до мене підійш{стать:ов:ла}? Забирайся.,"Вау! Це ідеально!\n\nА знаєте, здавалося б, Ві...",

In [47]:
untranslated[untranslated['title_uk'].notnull()].to_pickle('translations_df.pkl')

In [8]:
translations_df = pd.read_pickle('translations_df.pkl')

In [37]:
from crowdin_api import CrowdinClient

CROWDIN_PROJECT_ID=393919
crowdin_client = CrowdinClient(token=os.getenv('CROWDIN_TOKEN'), project_id=CROWDIN_PROJECT_ID, )

In [4]:
crowdin_files = dict()
offset = 0
page_size = 500
while (True):
    files = crowdin_client.source_files.list_files(CROWDIN_PROJECT_ID, offset=offset, limit=page_size)
    if not files['data']:
        break
    for file in files['data']:
        file_id = file['data']['id']
        file_path = file['data']['path']
        crowdin_files[file_path] = file_id
    offset += page_size

crowdin_files

{'/quests/Battlegrounds/Alterac Valley/A Gallon of Blood_7385.xml': 8826,
 '/quests/Battlegrounds/Alterac Valley/Ally of the Tauren_7362.xml': 8830,
 '/quests/Battlegrounds/Alterac Valley/Alterac Valley Graveyards_7081.xml': 8834,
 '/quests/Battlegrounds/Alterac Valley/Armor Scraps_7223.xml': 8838,
 '/quests/Battlegrounds/Alterac Valley/Begin the Attack_6846.xml': 8840,
 '/quests/Battlegrounds/Alterac Valley/Brotherly Love_7281.xml': 8842,
 '/quests/Battlegrounds/Alterac Valley/Brotherly Love_7282.xml': 8844,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Guses Fleet_6825.xml': 8846,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Ichmans Fleet_6943.xml': 8848,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Jeztors Fleet_6826.xml': 8850,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Mulvericks Fleet_6827.xml': 8852,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Slidores Fleet_6942.xml': 8854,
 '/quests/Battlegrounds/Alterac Valley/Call of Air - Vipores Fl

In [28]:
def get_quest_filename(row):
    quest_id = row['id']
    quest_title = row['title_en']
    valid_chars = frozenset('-.() abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    return ''.join(c for c in quest_title if c in valid_chars) + '_' + str(quest_id)

def get_path_for_quest(row):
    path = '/quests' + (f"_{row['expansion']}" if row['expansion'] != 'classic' else '')
    filename = get_quest_filename(row)
    path = path + f"/{row['cat']}/{filename}.xml"
    return path

def upload_translation(string_data, translation_text):
    crowdin_client.source_strings.get_string(string_data['id'])
    crowdin_client.string_translations.add_translation(stringId=string_data['id'], languageId='uk', text=translation_text)

def upload_translations(row):
    path = get_path_for_quest(row)
    file_id = crowdin_files.get(path)
    if not file_id:
        print(f"File #{file_id} not found for {path}.")
        return
    strings = crowdin_client.source_strings.list_strings(fileId=file_id)['data']
    strings_by_key = {string_data['data']['identifier']: string_data['data'] for string_data in strings}
    upload_translation(strings_by_key['TITLE'], row['title_uk'])
    if strings_by_key.get('OBJECTIVE') and row['objective_uk']:
        upload_translation(strings_by_key['OBJECTIVE'], row['objective_uk'])
    if strings_by_key.get('DESCRIPTION') and row['description_uk']:
        upload_translation(strings_by_key['DESCRIPTION'], row['description_uk'])
    if strings_by_key.get('PROGRESS') and row['progress_uk']:
        upload_translation(strings_by_key['PROGRESS'], row['progress_uk'])
    if strings_by_key.get('COMPLETION') and row['completion_uk']:
        upload_translation(strings_by_key['COMPLETION'], row['completion_uk'])



In [34]:
len(list(translations_df.iterrows()))

115

In [29]:
for _, row in translations_df.iterrows():
    filename = get_quest_filename(row)
    try:
        upload_translations(row)
        print(f"Uploaded translations for {filename}")
    except Exception as e:
        print(f"Error uploading translations for {filename}: {e}")

Error uploading translations for In a Bind_83808: ValidationError: http_status=400, request_id=None, detail: None, context=b'{"errors":[{"error":{"key":"translation","errors":[{"code":"validationError","message":"An identical translation of this string has been already saved."}]}}]}'
Error uploading translations for The Fallen Knight_83822: ValidationError: http_status=400, request_id=None, detail: None, context=b'{"errors":[{"error":{"key":"translation","errors":[{"code":"validationError","message":"An identical translation of this string has been already saved."}]}}]}'
Error uploading translations for A Lesson in Violence_83823: ValidationError: http_status=400, request_id=None, detail: None, context=b'{"errors":[{"error":{"key":"translation","errors":[{"code":"validationError","message":"An identical translation of this string has been already saved."}]}}]}'
Error uploading translations for Show Me The Money_83934: ValidationError: http_status=400, request_id=None, detail: None, con